In [38]:
!pip install gymnasium

In [39]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [40]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [41]:
env.P[0][3] # Transition model

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [42]:
env.observation_space.n

16

In [43]:
env.action_space.n

4

In [44]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [45]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 18)

In [46]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 77)

In [47]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 6)

In [48]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 4)

In [49]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 26)

In [50]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [51]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [52]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 57/1000
Average number of steps: 12.421052631578947


In [53]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 101/1000
Average number of steps: 16.475247524752476


In [54]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 770/1000
Average number of steps: 41.6


In [96]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            # print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [56]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [57]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [58]:
np.all(v_values_1 >= v_values_0)

True

In [59]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [60]:
np.all(v_values_2 >= v_values_1)

True

In [61]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [62]:
np.all(v_values_3 >= v_values_2)

True

In [81]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)
    step = 0

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            # print(f'Converged at {i}-th iteration.')
            step = i
            break

    return v_values, step

In [64]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [65]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [66]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [67]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [68]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [69]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 34)

In [70]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 743/1000
Average number of steps: 43.427994616419916


# My code

In [94]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    pi_values = np.zeros(env.observation_space.n)
    step = 0

    for i in range(max_iters):
        prev_pi_values = np.copy(pi_values)

        # Policy evaluation
        v_values = policy_evaluation(env, pi_values)

        # Polucy improvement
        pi_values = policy_extraction(env, v_values)

        # Check convergence
        if np.array_equal(pi_values, prev_pi_values):
            step = i
            break

    return pi_values, step

In [83]:
def myValueIteration(env, n_iter=50):
    runTimes = []
    steps = []
    for i in range(n_iter):
        start = time.time()

        # Value iteration
        optimal_v_values, step = value_iteration(env, max_iters=500, gamma=0.9)

        # Policy Extraction
        optimal_pi_values = policy_extraction(env, optimal_v_values, gamma=0.9)

        end = time.time()

        runTimes.append(end - start)
        steps.append(step)

    return runTimes, steps

In [91]:
def myPolicyIteration(env, n_iter=50):
    runTimes = []
    steps = []

    for i in range(n_iter):
        start = time.time()

        # Policy iteration
        optimal_pi_values, step = policy_iteration(env, max_iters=500, gamma=0.9)

        end = time.time()

        runTimes.append(end - start)
        steps.append(step)

    return runTimes, steps

# FrozenLake-v1

In [87]:
# Create environment
env = gym.make('FrozenLake-v1')

In [88]:
# Value iteration
frozenLake_value_times, frozenLake_value_steps = myValueIteration(env)

In [97]:
# Policy iteration
frozenLake_policy_times, frozenLake_policy_steps = myPolicyIteration(env)

In [100]:
print(f"Average runtime of Value Iteration: {np.average(frozenLake_value_times)} sec")
print(f"Average runtime of Policy Iteration: {np.average(frozenLake_policy_times)} sec")

Average runtime of Value Iteration: 0.11993456840515136 sec
Average runtime of Policy Iteration: 0.08793222427368164 sec


# FrozenLake8x8-v1


In [101]:
env = gym.make("FrozenLake8x8-v1")

In [102]:
# Value iteration
frozenLake8x8_value_times, frozenLake8x8_value_steps = myValueIteration(env)

In [103]:
# Policy iteration
frozenLake8x8_policy_times, frozenLake8x8_policy_steps = myPolicyIteration(env)

In [104]:
print(f"Average runtime of Value Iteration: {np.average(frozenLake8x8_value_times)} sec")
print(f"Average runtime of Policy Iteration: {np.average(frozenLake8x8_policy_times)} sec")

Average runtime of Value Iteration: 0.6339145755767822 sec
Average runtime of Policy Iteration: 1.0489384841918945 sec


# Taxi-v3

In [105]:
env = gym.make('Taxi-v3')

In [106]:
# Value iteration
taxi_value_times, taxi_value_steps = myValueIteration(env)

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


In [107]:
# Policy iteration
taxi_policy_times, taxi_policy_steps = myPolicyIteration(env)

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


In [108]:
print(f"Average runtime of Value Iteration: {np.average(taxi_value_times)} sec")
print(f"Average runtime of Policy Iteration: {np.average(taxi_policy_times)} sec")

Average runtime of Value Iteration: 5.9809559631347655 sec
Average runtime of Policy Iteration: 12.996360387802124 sec


# Nhận xét

In [123]:
print("\t\t\tAverage runtime of Value Iteration\tAverage runtime of Policy Iteration\n")
print(f"FrozenLake-v1\t\t {np.average(frozenLake_value_times)} \t\t\t {np.average(frozenLake_policy_times)}\n")
print(f"FrozenLake8x8-v1\t {np.average(frozenLake8x8_value_times)} \t\t\t {np.average(frozenLake8x8_policy_times)}\n")
print(f"Taxi-v3\t\t\t {np.average(taxi_value_times)} \t\t\t {np.average(taxi_policy_times)}\n")

print("------------------------------------------------------------------------------------------------------------------")
print("\t\t\tAverage step of Value Iteration \t Average step of Policy Iteration\n")
print(f"FrozenLake-v1\t\t {np.average(frozenLake_value_steps)} \t\t\t\t\t {np.average(frozenLake_policy_steps)}\n")
print(f"FrozenLake8x8-v1\t {np.average(frozenLake8x8_value_steps)} \t\t\t\t\t {np.average(frozenLake8x8_policy_steps)}\n")
print(f"Taxi-v3\t\t\t {np.average(taxi_value_steps)} \t\t\t\t\t {np.average(taxi_policy_steps)}\n")

			Average runtime of Value Iteration	Average runtime of Policy Iteration

FrozenLake-v1		 0.11993456840515136 			 0.08793222427368164

FrozenLake8x8-v1	 0.6339145755767822 			 1.0489384841918945

Taxi-v3			 5.9809559631347655 			 12.996360387802124

------------------------------------------------------------------------------------------------------------------
			Average step of Value Iteration 	 Average step of Policy Iteration

FrozenLake-v1		 79.0 					 5.0

FrozenLake8x8-v1	 117.0 					 9.0

Taxi-v3			 116.0 					 16.0



In [ ]:
"""
Với trung bình 50 lần chạy:
- Trong các toy games 'FrozenLake-v1', 'FrozenLake8x8-v1', và 'Taxi-v3', thuật toán Value iteration đều cho ra kết quả nhanh hơn thuật toán Policy Iteration.
Đặc biệt, trong games Taxi-v3, thuật toán Value iteration chạy nhanh hơn Policy iteration ~2 lần.

- Trong các toy games ở trên, có thể thấy thuật toán Policy Iteration mất rất ít vòng lặp để hội tụ, thế nhưng lại mất rất nhiều thời gian để cho ra kết quả đó.
Điều này cho thấy rằng với mỗi chiến lược, thuật toán Policy tốn rất nhiều thời gian để đánh giá và cải thiện chúng.

- Tóm lại, ta có kết luận sau: thuật toán Value iteration mất rất nhiều bước để hội tụ, nhưng thời gian tính toán lại nhanh hơn so với thuật toán Policy Iteration.
"""